# **IMPORT INTO RDF**

In [1]:
import pandas as pd

In [2]:
#The variable where we will store all the new rdf content
text_final="" 

# Dataset for THEATER

## Cleaning

In [3]:
dft = pd.read_csv("etablissements-cinematographiques.csv", sep=";")
dft.head(3)

,N° auto,nom,région administrative,adresse,code INSEE,commune,population de la commune,DEP,N°UU,unité urbaine,...,nombre de films en semaine 1,PdM en entrées des films français,PdM en entrées des films américains,PdM en entrées des films européens,PdM en entrées des autres films,films Art et Essai,PdM en entrées des films Art et Essai,latitude,longitude,geolocalisation
0,54,GAUMONT CHAMPS ELYSEES MARIGNAN,ILE-DE-FRANCE,27/33 AVENUE DES CHAMPS ELYSEES,75108,Paris 8e Arrondissement,36827.0,75,00851,Paris,...,66.0,22.077070,55.579168,20.830900,1.512862,9.0,7.583575,48.869654,2.306873,"48.869654, 2.306873"
1,311,GRAND REX,ILE-DE-FRANCE,5 BD POISSONNIERE,75102,Paris 2e Arrondissement,21561.0,75,00851,Paris,...,78.0,16.722357,66.827151,10.889319,5.561173,5.0,10.080847,48.868720,2.347608,"48.86872, 2.347608"
2,451,UGC MONTPARNASSE,ILE-DE-FRANCE,83 BD DU MONTPARNASSE,75106,Paris 6e Arrondissement,41384.0,75,00851,Paris,...,74.0,46.697245,37.310727,15.992028,0.000000,18.0,28.506839,48.843510,2.325646,"48.84351, 2.325646"


In [4]:
dft_new = pd.DataFrame({'nom':dft['nom']})
dft_new = dft_new.merge(dft[['adresse','nom']])
dft_new = dft_new.merge(dft[['commune','nom']])
dft_new = dft_new.assign(new_col="France")
dft_new.columns = ['Name', 'Location','City', 'Country']
dft_new = dft_new.head(3)

## Text creation

In [5]:
indiv = "\n\n";
col = []
for index, row in dft_new.iterrows():
    name = row['Name']
    name += "_"+row['City']
    name = name.replace("&", "and")
    location = row['Location']
    location = location.replace("&", "and")
    country = row['Country']
    individual = name.replace(" ", "_")
    col.append(individual) #retrieve value for schedule
    
    text = f'    <!-- http://www.semanticweb.org/kiouloueleonor/ontologies/2023/2/untitled-ontology-16#{individual} -->\n\n'
    text += f'    <owl:NamedIndividual rdf:about="http://www.semanticweb.org/kiouloueleonor/ontologies/2023/2/untitled-ontology-16#{individual}">\n'
    text += f'\t<rdf:type rdf:resource="http://www.semanticweb.org/kiouloueleonor/ontologies/2023/2/untitled-ontology-16#Theater"/>\n'
    text += f'\t<untitled-ontology-16:Country>{country}</untitled-ontology-16:Country>\n'
    text += f'\t<untitled-ontology-16:Location>{location}</untitled-ontology-16:Location>\n'
    text += f'\t<untitled-ontology-16:Name>{name}</untitled-ontology-16:Name>\n'
    text += '    </owl:NamedIndividual>'
    
    print(text)
    indiv += "\n\n" + text + "\n\n"
dft_new = dft_new.assign(indiv_name = col)
text_final += indiv

    <!-- http://www.semanticweb.org/kiouloueleonor/ontologies/2023/2/untitled-ontology-16#GAUMONT_CHAMPS_ELYSEES_MARIGNAN_Paris_8e_Arrondissement -->

    <owl:NamedIndividual rdf:about="http://www.semanticweb.org/kiouloueleonor/ontologies/2023/2/untitled-ontology-16#GAUMONT_CHAMPS_ELYSEES_MARIGNAN_Paris_8e_Arrondissement">
	<rdf:type rdf:resource="http://www.semanticweb.org/kiouloueleonor/ontologies/2023/2/untitled-ontology-16#Theater"/>
	<untitled-ontology-16:Country>France</untitled-ontology-16:Country>
	<untitled-ontology-16:Location>27/33 AVENUE DES CHAMPS ELYSEES</untitled-ontology-16:Location>
	<untitled-ontology-16:Name>GAUMONT CHAMPS ELYSEES MARIGNAN_Paris 8e Arrondissement</untitled-ontology-16:Name>
    </owl:NamedIndividual>
    <!-- http://www.semanticweb.org/kiouloueleonor/ontologies/2023/2/untitled-ontology-16#GRAND_REX_Paris_2e_Arrondissement -->

    <owl:NamedIndividual rdf:about="http://www.semanticweb.org/kiouloueleonor/ontologies/2023/2/untitled-ontology-16#GRAND_R

In [6]:
dft_new

,Name,Location,City,Country,indiv_name
0,GAUMONT CHAMPS ELYSEES MARIGNAN,27/33 AVENUE DES CHAMPS ELYSEES,Paris 8e Arrondissement,France,GAUMONT_CHAMPS_ELYSEES_MARIGNAN_Paris_8e_Arron...
1,GRAND REX,5 BD POISSONNIERE,Paris 2e Arrondissement,France,GRAND_REX_Paris_2e_Arrondissement
2,UGC MONTPARNASSE,83 BD DU MONTPARNASSE,Paris 6e Arrondissement,France,UGC_MONTPARNASSE_Paris_6e_Arrondissement


# Dataset for PERSON , MOVIE, SCHEDULE

## Cleaning

In [7]:
df = pd.read_csv("new_data.csv", sep=';')
df.head(3)

,movie_id,title,genres,release_date,cast,cast_complet,production_companies,budget,popularity,spoken_languages,vote_average
0,19995,Avatar,"['Action', 'Adventure', 'Fantasy', 'Science Fi...",2009-12-10,"['Sam Worthington', 'Zoe Saldana', 'Sigourney ...","['Sam Worthington', 'Zoe Saldana', 'Sigourney ...","['Ingenious Film Partners', 'Twentieth Century...",237000000,150.437577,"['en', 'es']",7.2
1,285,Pirates of the Caribbean: At World's End,"['Adventure', 'Fantasy', 'Action']",2007-05-19,"['Johnny Depp', 'Orlando Bloom', 'Keira Knight...","['Johnny Depp', 'Orlando Bloom', 'Keira Knight...","['Walt Disney Pictures', 'Jerry Bruckheimer Fi...",300000000,139.082615,['en'],6.9
2,206647,Spectre,"['Action', 'Adventure', 'Crime']",2015-10-26,"['Daniel Craig', 'Christoph Waltz', 'Léa Seydo...","['Daniel Craig', 'Christoph Waltz', 'Léa Seydo...","['Columbia Pictures', 'Danjaq', 'B24']",245000000,107.376788,"['fr', 'en', 'es', 'it', 'de']",6.3


Drop the columns and movies we don't need

In [8]:
df = df.drop(columns=['movie_id','cast_complet', 'production_companies', 'budget', 'popularity', 'vote_average','genres', 'spoken_languages'])
df = df.head(3)

Creation of the columns we need

In [9]:
#Retrive the years 
liste_years = df['release_date']
years = []
for string in liste_years:
    years.append(int(string[:4]))

In [10]:
df = df.assign(Country="USA")
df = df.assign(genres="Action")
df = df.assign(spoken_languages	="English")
df = df.assign(Years=years)

In [11]:
date_fin = ['2010-02-24','2007-09-19','2016-01-07']
prog = ['schedule_Avatar', 'schedule_POTC', 'schedule_Spectre']
df = df.assign(end_date=date_fin)
df = df.assign(schedule=prog)

In [12]:
df

,title,release_date,cast,Country,genres,spoken_languages,Years,end_date,schedule
0,Avatar,2009-12-10,"['Sam Worthington', 'Zoe Saldana', 'Sigourney ...",USA,Action,English,2009,2010-02-24,schedule_Avatar
1,Pirates of the Caribbean: At World's End,2007-05-19,"['Johnny Depp', 'Orlando Bloom', 'Keira Knight...",USA,Action,English,2007,2007-09-19,schedule_POTC
2,Spectre,2015-10-26,"['Daniel Craig', 'Christoph Waltz', 'Léa Seydo...",USA,Action,English,2015,2016-01-07,schedule_Spectre


## SCHEDULE

In [279]:
import random
def rdm_theater(liste):
    return random.sample(liste, 2)

In [280]:
list_theater = list(dft_new['indiv_name'])
list_theater.append('CGR_Nanterre')
list_theater.append('Pathé_Conflans')
print(list_theater)

['GAUMONT_CHAMPS_ELYSEES_MARIGNAN_Paris_8e_Arrondissement', 'GRAND_REX_Paris_2e_Arrondissement', 'UGC_MONTPARNASSE_Paris_6e_Arrondissement', 'CGR_Nanterre', 'Pathé_Conflans']


In [281]:
import time
indiv = "\n\n";
for index, row in df.iterrows():
    individual = row['schedule']
    start = row['release_date']
    end = row['end_date']
    theater = rdm_theater(list_theater)
    
    text = f'    <!-- http://www.semanticweb.org/kiouloueleonor/ontologies/2023/2/untitled-ontology-16#{individual} -->\n\n'
    text += f'    <owl:NamedIndividual rdf:about="http://www.semanticweb.org/kiouloueleonor/ontologies/2023/2/untitled-ontology-16#{individual}">\n'
    text += f'\t<rdf:type rdf:resource="http://www.semanticweb.org/kiouloueleonor/ontologies/2023/2/untitled-ontology-16#MovieSchedule"/>\n'
    text += f'\t<untitled-ontology-16:isPlayedIn rdf:resource="http://www.semanticweb.org/kiouloueleonor/ontologies/2023/2/untitled-ontology-16#{theater[0]}"/>n'
    text += f'\t<untitled-ontology-16:isPlayedIn rdf:resource="http://www.semanticweb.org/kiouloueleonor/ontologies/2023/2/untitled-ontology-16#{theater[1]}"/>n'
    text += f'\t<untitled-ontology-16:endDate>{end}</untitled-ontology-16:endDate>\n'
    text += f'\t<untitled-ontology-16:startDate>{start}</untitled-ontology-16:startDate>\n'
    text += '    </owl:NamedIndividual>'
    
    #print(text)
    indiv += "\n\n" + text + "\n\n"
    time.sleep(1)
text_final += indiv

## PERSON

Retrieve all the Actors

In [282]:
acteurs = df['cast']
liste_acteurs = []
list_act1 = []
for sublist in acteurs:
    list_act1.append(sublist)
    for item in sublist:
        liste_acteurs.append(item)

Modification of the format of the list

In [283]:
new_list = []
for item in list_act1:
    eval_list = eval(item)
    for name in eval_list:
        new_list.append(name.strip("'"))
clean_list = [name.replace("'", "") for name in new_list]       
print(clean_list)

['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver', 'Johnny Depp', 'Orlando Bloom', 'Keira Knightley', 'Daniel Craig', 'Christoph Waltz', 'Léa Seydoux']


Retrieve the film played by each actors

In [284]:
liste1=clean_list[:3]
liste2=clean_list[3:6]
liste3=clean_list[-3:]
print(liste3)

['Daniel Craig', 'Christoph Waltz', 'Léa Seydoux']


In [285]:
def trad(liste, filmName):
    indiv = "\n\n";
    for i in liste:
        name = i
        individual = name.replace(" ", "_")
        filmName = filmName.replace(" ", "_")

        text = f'    <!-- http://www.semanticweb.org/kiouloueleonor/ontologies/2023/2/untitled-ontology-16#{individual} -->\n\n'
        text += f'    <owl:NamedIndividual rdf:about="http://www.semanticweb.org/kiouloueleonor/ontologies/2023/2/untitled-ontology-16#{individual}">\n'
        text += f'\t<rdf:type rdf:resource="http://www.semanticweb.org/kiouloueleonor/ontologies/2023/2/untitled-ontology-16#Person"/>\n'
        text += f'\t<untitled-ontology-16:isActorOf rdf:resource="http://www.semanticweb.org/kiouloueleonor/ontologies/2023/2/untitled-ontology-16#{filmName}"/>\n'
        text += f'\t<untitled-ontology-16:Name>{individual}</untitled-ontology-16:Name>\n'
        text += '    </owl:NamedIndividual>'

        #print(text)
        indiv += "\n\n" + text + "\n\n"
    return indiv


In [286]:
rdf_text = ""
rdf_text += trad(liste1, df['title'][0])
rdf_text += trad(liste2, df['title'][1])
rdf_text += trad(liste3, df['title'][2])
text_final += rdf_text

## MOVIE

In [287]:
indiv = "\n\n";
for index, row in df.iterrows():
    name = row['title']
    years = row['Years']
    Genres = row['genres']
    Schedule = row['schedule']
    Language = row['spoken_languages']
    country = row['Country']
    individual = name.replace(" ", "_")
    
    text = f'    <!-- http://www.semanticweb.org/kiouloueleonor/ontologies/2023/2/untitled-ontology-16#{individual} -->\n\n'
    text += f'    <owl:NamedIndividual rdf:about="http://www.semanticweb.org/kiouloueleonor/ontologies/2023/2/untitled-ontology-16#{individual}">\n'
    text += f'\t<rdf:type rdf:resource="http://www.semanticweb.org/kiouloueleonor/ontologies/2023/2/untitled-ontology-16#Movie"/>\n'
    text += f'\t<untitled-ontology-16:hasGenre rdf:resource="http://www.semanticweb.org/kiouloueleonor/ontologies/2023/2/untitled-ontology-16#{Genres}"/>\n'
    text += f'\t<untitled-ontology-16:hasSchedule rdf:resource="http://www.semanticweb.org/kiouloueleonor/ontologies/2023/2/untitled-ontology-16#{Schedule}"/>\n'
    text += f'\t<untitled-ontology-16:Country>{country}</untitled-ontology-16:Country>\n'
    text += f'\t<untitled-ontology-16:Language>{Language}</untitled-ontology-16:Language>\n'
    text += f'\t<untitled-ontology-16:Title>{name}</untitled-ontology-16:Title>\n'
    text += f'\t<untitled-ontology-16:Year rdf:datatype="http://www.w3.org/2001/XMLSchema#integer">{years}</untitled-ontology-16:Year>\n'
    text += '    </owl:NamedIndividual>'
    
    #print(text)
    indiv += "\n\n" + text + "\n\n"
indiv += '</rdf:RDF>\n\n\n\n'
indiv += "<!-- Generated by the OWL API (version 4.5.9.2019-02-01T07:24:44Z) https://github.com/owlcs/owlapi -->\n\n"
text_final += indiv

# IMPORT

In [289]:
with open("projet.owl", "r+b") as file:
    contenu = file.read()
    file.seek(-119, 1)
    file.write(text_final.encode())